In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def find_cols_from_substring(columns, keys):
    res = []
    for key in keys:
        p = re.compile(key)
        res.extend([s for s in columns if p.search(s)])
    return res

In [3]:
training_data = pd.read_csv("../datasets/pml-training.csv", index_col="Unnamed: 0")
testing_data = pd.read_csv("../datasets/pml-testing.csv", index_col="Unnamed: 0")

/home/tasos51/miniconda3/envs/ML4QS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (11,14,19,22,25,70,73,86,87,89,90,94,97,100) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
keys = ['kurtosis', 'skewness', 'max', 'min', 'amplitude', 'var', 'avg', 'stddev']

training_data['raw_timestamp'] = training_data['raw_timestamp_part_1'].astype(str) + \
                                "." + \
                                training_data['raw_timestamp_part_2'].astype(str)

training_data['raw_timestamp'] = (pd.to_numeric(training_data['raw_timestamp']) * 10**9).astype('int64')
training_data['raw_timestamp'] = pd.to_datetime(training_data['raw_timestamp'], unit = 'ns')
training_data.set_index('raw_timestamp', inplace=True)
training_data = training_data.sort_values(by='raw_timestamp', axis=0)


training_data = pd.concat([training_data, pd.get_dummies(training_data['classe'], prefix='label', prefix_sep='')], axis=1)

unused_cols = ['raw_timestamp_part_1', 'raw_timestamp_part_2', 'classe', 'new_window', 'num_window', 'cvtd_timestamp']
unused_cols.extend(find_cols_from_substring(training_data.columns, keys))
training_data.drop(unused_cols, axis=1, inplace=True)

training_data
training_data.to_csv('../datasets/training-processed-combined.csv')

In [5]:
groups = training_data.groupby('user_name')

for name, group in groups:
    g = group.copy()
    g.drop('user_name', axis=1, inplace=True)
    g.to_csv('../datasets/' + name + '.csv')